In [ ]:
# --- 설치 ---
!pip -q install konlpy mecab-python3 sentencepiece pandas scikit-learn


[알림] Mecab 사용 불가 → Okt로 진행합니다.
사용 형태소 분석기: okt
[whitespace] [('어제 비가 왔지만 우리는 야외 공연을 강행했다.', 0.4364), ('콘서트가 예정대로 진행됐다는 뉴스가 보도됐다.', 0.2582), ('한국어는 조사·어미 때문에 영어보다 토큰화가 어렵다.', 0.0), ('SNS 텍스트 전처리에는 이모지와 줄임말 처리가 중요하다.', 0.0), ('Mecab은 빠르고 정확하지만 설치가 까다롭다.', 0.0)]
[regex_ko] [('어제 비가 왔지만 우리는 야외 공연을 강행했다.', 0.4364), ('콘서트가 예정대로 진행됐다는 뉴스가 보도됐다.', 0.2582), ('한국어는 조사·어미 때문에 영어보다 토큰화가 어렵다.', 0.0), ('SNS 텍스트 전처리에는 이모지와 줄임말 처리가 중요하다.', 0.0), ('Mecab은 빠르고 정확하지만 설치가 까다롭다.', 0.0)]
[morph   ] [('콘서트가 예정대로 진행됐다는 뉴스가 보도됐다.', 0.5657), ('어제 비가 왔지만 우리는 야외 공연을 강행했다.', 0.1751), ('한국어는 조사·어미 때문에 영어보다 토큰화가 어렵다.', 0.0), ('SNS 텍스트 전처리에는 이모지와 줄임말 처리가 중요하다.', 0.0), ('Mecab은 빠르고 정확하지만 설치가 까다롭다.', 0.0)]
[sp      ] [('콘서트가 예정대로 진행됐다는 뉴스가 보도됐다.', 0.6895), ('어제 비가 왔지만 우리는 야외 공연을 강행했다.', 0.5661), ('형태소 기반으로 명사와 동사를 중점적으로 본다.', 0.3069), ('Mecab은 빠르고 정확하지만 설치가 까다롭다.', 0.3022), ('서브워드 방식은 OOV 문제를 완화한다.', 0.2617)]


In [4]:

from konlpy.tag import Okt
import sentencepiece as spm
import re, numpy as np, pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

# --- 형태소 준비(Mecab→Okt 폴백) ---
tokenizer_name = "okt"; okt = Okt(); mecab=None
try:
    from konlpy.tag import Mecab
    mecab = Mecab(); tokenizer_name = "mecab"
except:
    print("[알림] Mecab 사용 불가 → Okt로 진행합니다.")
print("사용 형태소 분석기:", tokenizer_name)

def tok_ws(text): return text.split()
def tok_regex_ko(text):
    s = re.sub(r"[^가-힣A-Za-z0-9\s]", " ", text)
    s = re.sub(r"\s+", " ", s).strip()
    return s.split()
def tok_morph(text):
    if tokenizer_name=="mecab": return [t for t in mecab.morphs(text) if len(t)>=2]
    else: return [t for t in okt.morphs(text, stem=True) if len(t)>=2]

# --- SentencePiece 학습(간단 Unigram) ---
CORPUS = [
    "오늘은 비가 왔지만 콘서트는 예정대로 진행됐다.",
    "신조어와 줄임말이 많은 SNS 텍스트는 전처리가 중요하다.",
    "한국어 토큰화는 조사와 어미 때문에 어렵다.",
    "메캅은 빠르고 정확하다는 평가가 많다.",
    "서브워드는 OOV를 줄이는 데 유리하다.",
    "텍스트 마이닝에서는 불용어 제거가 자주 사용된다.",
    "형태소 분석으로 품사 정보를 활용할 수 있다.",
]
open("corpus.txt","w",encoding="utf-8").write("\n".join(CORPUS))
spm.SentencePieceTrainer.Train(
    input="corpus.txt",
    model_prefix="sp_search",
    model_type="unigram",
    vocab_size=97, # Reduced vocab_size
    character_coverage=0.9995
)
sp = spm.SentencePieceProcessor(); sp.load("sp_search.model")
def tok_sp(text): return sp.EncodeAsPieces(text)

# --- 검색 코퍼스(문서들) ---
DOCS = [
    "어제 비가 왔지만 우리는 야외 공연을 강행했다.",
    "SNS 텍스트 전처리에는 이모지와 줄임말 처리가 중요하다.",
    "한국어는 조사·어미 때문에 영어보다 토큰화가 어렵다.",
    "Mecab은 빠르고 정확하지만 설치가 까다롭다.",
    "서브워드 방식은 OOV 문제를 완화한다.",
    "형태소 기반으로 명사와 동사를 중점적으로 본다.",
    "전처리에서 불용어를 제거하면 잡음이 줄어든다.",
    "콘서트가 예정대로 진행됐다는 뉴스가 보도됐다.",
]

# --- 토크나이저별 TF-IDF 검색 ---
def search(query, docs, tok_fn):
    vec = TfidfVectorizer(token_pattern=r"[^ ]+")
    X = vec.fit_transform(" ".join(tok_fn(d)) for d in docs)
    q = vec.transform([" ".join(tok_fn(query))])
    sims = (q @ X.T).toarray().ravel()
    idx = np.argsort(-sims)
    return [(docs[i], round(float(sims[i]),4)) for i in idx[:5]]

query = "비가 왔지만 콘서트는 예정대로 진행"
print("[whitespace]", search(query, DOCS, tok_ws))
print("[regex_ko]",  search(query, DOCS, tok_regex_ko))
print("[morph   ]",  search(query, DOCS, tok_morph))
print("[sp      ]",  search(query, DOCS, tok_sp))

사용 형태소 분석기: mecab
[whitespace] [('어제 비가 왔지만 우리는 야외 공연을 강행했다.', 0.4364), ('콘서트가 예정대로 진행됐다는 뉴스가 보도됐다.', 0.2582), ('SNS 텍스트 전처리에는 이모지와 줄임말 처리가 중요하다.', 0.0), ('한국어는 조사·어미 때문에 영어보다 토큰화가 어렵다.', 0.0), ('Mecab은 빠르고 정확하지만 설치가 까다롭다.', 0.0)]
[regex_ko] [('어제 비가 왔지만 우리는 야외 공연을 강행했다.', 0.4364), ('콘서트가 예정대로 진행됐다는 뉴스가 보도됐다.', 0.2582), ('SNS 텍스트 전처리에는 이모지와 줄임말 처리가 중요하다.', 0.0), ('한국어는 조사·어미 때문에 영어보다 토큰화가 어렵다.', 0.0), ('Mecab은 빠르고 정확하지만 설치가 까다롭다.', 0.0)]
[morph   ] [('콘서트가 예정대로 진행됐다는 뉴스가 보도됐다.', 0.6972), ('어제 비가 왔지만 우리는 야외 공연을 강행했다.', 0.1356), ('Mecab은 빠르고 정확하지만 설치가 까다롭다.', 0.1356), ('SNS 텍스트 전처리에는 이모지와 줄임말 처리가 중요하다.', 0.0), ('한국어는 조사·어미 때문에 영어보다 토큰화가 어렵다.', 0.0)]
[sp      ] [('콘서트가 예정대로 진행됐다는 뉴스가 보도됐다.', 0.6895), ('어제 비가 왔지만 우리는 야외 공연을 강행했다.', 0.5661), ('형태소 기반으로 명사와 동사를 중점적으로 본다.', 0.3069), ('Mecab은 빠르고 정확하지만 설치가 까다롭다.', 0.3022), ('서브워드 방식은 OOV 문제를 완화한다.', 0.2617)]


sentencepiece_trainer.cc(78) LOG(INFO) Starts training with : 
trainer_spec {
  input: corpus.txt
  input_format: 
  model_prefix: sp_search
  model_type: UNIGRAM
  vocab_size: 97
  self_test_sample_size: 0
  character_coverage: 0.9995
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  pretokenization_delimiter: 
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  seed_sentencepieces_file: 
  hard_vocab_limit: 1
  use_all_vocab: 0
  unk_id: 0
  bos_id: 1
  eos_id: 2
  pad_id: -1
  unk_piece: <unk>
  bos_piece: <s>
  eos_piece: </s>
  pad_piece: <pad>
  unk_surface:  ⁇ 
  enable_differential_privacy: 0
  differen